In [11]:
import os
import vllm
import json
from easysteer.hidden_states import get_all_hidden_states
from vllm import LLM

from easysteer.steer import extract_pca_control_vector,StatisticalControlVector


os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["VLLM_USE_V1"] = "0"

# file_path = "../../temp/test.jsonl" #MATH-500
file_path = "../../temp/test_gsm8k.jsonl" #GSM8K

model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"

# vector_path = "vectors/thinking_switch_pca_MATH-500.gguf" #MATH-500
vector_path = "GSM8K.gguf"

num_question = 10

In [12]:
problem_list = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        line_count = 0
        for line in f:
            if line_count >= num_question:
                break
            stripped_line = line.strip()
            if not stripped_line:
                continue
                
            try:
                data = json.loads(stripped_line)
                if "problem" in data:
                    problem_list.append(data["problem"])
                    line_count += 1
                elif "question" in data:
                    problem_list.append(data["question"])
                    line_count += 1
            except json.JSONDecodeError:
                print(f"skip: {line[:50]}...")
                continue

except Exception as e:
    print(f"{str(e)}")
# problem_list = ["Find the roots of $(x - 3)^3 + (x -7)^3 = (2x - 10)^3.$","A regular hexagon can be divided into six equilateral triangles. If the perimeter of one of the triangles is 21 inches, what is the perimeter, in inches, of the regular hexagon?",""]

In [13]:
slow_thinking = [
    "Please give the detailed thinking progress.",
    "Output the complete thinking steps.",
    "Walk me through your reasoning process step by step.",
    "Show me your detailed analysis and thought process.",
    "Please break down your thinking into clear steps.",
    "I want to see your complete reasoning chain.",
    "Explain your approach and methodology thoroughly.",
    "Provide a comprehensive breakdown of your solution process.",
    "Show me how you arrived at this conclusion with full details.",
    "Give me the complete analytical framework you used."
]

fast_thinking = [
    "Output the answer directly.",
    "Just give me the answer.",
    "Skip the explanation, just provide the result.",
    "Cut to the chase - what's the answer?",
    "Give me the bottom line immediately.",
    "No need for details, just the final answer.",
    "Straight to the point please.",
    "Just the conclusion, no process needed.",
    "Direct answer only.",
    "Quick response - answer first."
]

len_slow=len(slow_thinking)
len_fast=len(fast_thinking)

In [15]:
texts = []
for prob in problem_list:
    for req in slow_thinking+fast_thinking:
        texts.append(f"""<|im_start|>user
{prob}{req}<|im_end|>
<|im_start|>assistant
""")

llm = LLM(model=model_path,task="reward",tensor_parallel_size=1)

all_hidden_states, outputs = get_all_hidden_states(llm, texts)


INFO 07-12 19:04:19 [config.py:1472] Using max model len 32768
INFO 07-12 19:04:19 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-12 19:04:19 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend='')

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.49it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.48it/s]


INFO 07-12 19:04:21 [default_loader.py:272] Loading weights took 0.77 seconds
INFO 07-12 19:04:22 [model_runner.py:1255] Model loading took 2.8788 GiB and 0.824770 seconds



Processed prompts: 100%|██████████| 200/200 [00:00<00:00, 216.39it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


In [16]:

control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states,
    positive_indices=list(range(num_question*len_slow)), 
    negative_indices=list(range(num_question*len_slow,num_question*(len_slow+len_fast))),
    model_type="qwen2.5",
    token_pos=-1,
    normalize=True
)

Computing PCA directions: 100%|██████████| 28/28 [00:00<00:00, 241.81it/s]


In [17]:
control_vector.export_gguf(vector_path)
control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([ 0.00432168, -0.02199909, -0.02321268, ..., -0.00344263,
        -0.01230501,  0.00480626], shape=(1536,), dtype=float32), 1: memmap([ 0.01987046,  0.03280366,  0.03120032, ..., -0.02368647,
         0.01520148, -0.01653092], shape=(1536,), dtype=float32), 2: memmap([-0.00510607,  0.01795737, -0.03604069, ..., -0.04757744,
         0.04633784, -0.01485594], shape=(1536,), dtype=float32), 3: memmap([-0.01489841, -0.01592376, -0.03618836, ...,  0.0018658 ,
         0.01329398, -0.00049858], shape=(1536,), dtype=float32), 4: memmap([ 0.02282213,  0.00764716, -0.02953233, ..., -0.00735084,
         0.01396071,  0.00707858], shape=(1536,), dtype=float32), 5: memmap([ 0.01595395,  0.06010181, -0.03764697, ..., -0.01724483,
        -0.02025383, -0.04356156], shape=(1536,), dtype=float32), 6: memmap([ 0.01427394,  0.00484354, -0.05836817, ...,  0.01964263,
        -0.01654702, -0.00549133], shape=(1536,), dtype